In [2]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re
import os
import sqlite3

In [3]:
def read_docx(file_path):
    doc = Document(file_path)
    return doc

In [4]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [10]:
def df_find_cases(file_path):
    words_for_table = []  # Use a list to store repeated entries
    sujeito_list = [] # Initialize sujeito_list outside the loop
    curso_list = []  # Initialize curso_list outside the loop
    sexo_list = [] # Initialize sexo_list outside the loop
    ano_list = [] # Initialize ano_list outside the loop
    cidade_list = [] # Initialize cidade_list outside the loop

    # Iterate through all files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(directory_path, filename)
            text = read_docx(file_path)

            ### WORDS WITH PATTERN ###
            pattern = r'\S*\[[^\]]*(?:ɾ|ɹ|h|∅r)[^\]]*\]\S*'
            matches = re.findall(pattern, text)
            words_for_table.extend(matches)

            ###### SUJEITOS #######
            sujeito_match = re.search(r'\bSUJEITO:\s*(\S+)', text)
            if sujeito_match:
                sujeito_list.extend([sujeito_match.group(1)] * len(matches))
            else:
                sujeito_list.extend(['Not Specified'] * len(matches))
            
             ##### CURSO #########
            curso_match = re.search(r'\bCURSO:\s*(\S+)', text)
            if curso_match:
                curso_list.extend([curso_match.group(1)] * len(matches))
            else:
                curso_list.extend(['Not Specified'] * len(matches))

             ##### SEXO #######
            sexo_match = re.search(r'\bSEXO:\s*(\S+)', text)
            if sexo_match:
                sexo_list.extend([sexo_match.group(1)] * len(matches))
            else:
                sexo_list.extend(['Not Specified'] * len(matches))

            
             ####### ANO ########
            ano_match = re.search(r'\bANO:\s*(\S+)', text)
            if ano_match:
                ano_list.extend([ano_match.group(1)] * len(matches))
            else:
                ano_list.extend(['Not Specified'] * len(matches))

            ####### CIDADE ########
            cidade_match = re.search(r'\bCIDADE:\s*(\S+)', text)
            if cidade_match:
                cidade_list.extend([cidade_match.group(1)] * len(matches))
            else:
                cidade_list.extend(['Not Specified'] * len(matches))
    
    df = DataFrame({'palavra': words_for_table, 'sujeito': sujeito_list, 'curso': curso_list, 'sexo': sexo_list, 'ano': ano_list, 'cidade': cidade_list})


    #remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)
        
            
    return df

In [39]:
def create_rotico(row, i):
    pattern_1 = r'\[[^\]]*ɾ[^\]]*\]'
    pattern_2 = r'\[[^\]]*ɹ[^\]]*\]'
    pattern_3 = r'\[[^\]]*h[^\]]*\]'
    pattern_4 = r'\[[^\]]*∅r[^\]]*\]'

    pattern = pattern_1 + '|' + pattern_2 + '|' + pattern_3 + '|' + pattern_4
    
    
    rotico_cru = re.findall(pattern, row['palavra'])

    if row['rótico'] == '':
        if re.search(pattern_1, rotico_cru[i]):
            return '[ɾ]'
        if re.search(pattern_2, rotico_cru[i]):
            return '[ɹ]'
        if re.search(pattern_3, rotico_cru[i]):
            return '[h]'
        if re.search(pattern_4, rotico_cru[i]):
            return '[∅r]'
        
    return row['rótico']


In [96]:
def create_apagamento_rotico(row):

    if row['apagamento_rótico'] == '':    
        if row['rótico'] == '[∅r]':
                return 'Sim'
        else:
                return 'Não'

    return row['apagamento_rótico']

In [51]:
def create_rotico_cru(row, i):
    
    pattern_1 = r'\[[^\]]*ɾ[^\]]*\]'
    pattern_2 = r'\[[^\]]*ɹ[^\]]*\]'
    pattern_3 = r'\[[^\]]*h[^\]]*\]'
    pattern_4 = r'\[[^\]]*∅r[^\]]*\]'

    pattern = pattern_1 + '|' + pattern_2 + '|' + pattern_3 + '|' + pattern_4
    
    if row['rótico_cru'] == '':
        rotico_cru = re.findall(pattern, row['palavra'])
        pattern_n = r'\(n\)'
        pattern_v = r'\(v\)'
        pattern_nv = pattern_n + '|' + pattern_v
        rotico_cru[i] = re.sub(pattern, '', rotico_cru[i])
        return rotico_cru[i]
        
    return row['rótico_cru']

In [69]:
def create_verbo_ou_nome(row, i):
    
    pattern_n = r'\[[^\]]*\(n\)[^\]]*\]'
    pattern_v = r'\[[^\]]*\(v\)[^\]]*\]'

    pattern = pattern_n + '|' + pattern_v

    verbo_ou_nome_cru = re.findall(pattern, row['palavra'])
    
    if row['verbo_ou_nome'] == '':
        
        if re.search(pattern_n, verbo_ou_nome_cru[i]):
            return 'nome'
        if re.search(pattern_v, verbo_ou_nome_cru[i]):
            return 'verbo'    

    return row['verbo_ou_nome']

In [23]:
def create_dimensao_palavra(row):
    adjusted_length = 0
    inside_brackets = False

    for char in row:
        if char == '[':
            inside_brackets = True
            adjusted_length += 1
        elif char == ']':
            inside_brackets = False
        elif inside_brackets:
            # Inside brackets, count as one character
            continue
        else:
            # Outside brackets, count as individual characters
            adjusted_length += 1

    if adjusted_length < 5:
        dimensao = 'PEQUENO'
    elif adjusted_length > 4 and adjusted_length < 8:
        dimensao = 'MÉDIO'
    elif adjusted_length > 7:
        dimensao = 'GRANDE'

    return dimensao

In [56]:
def create_posicao_rotico(row):
    
    
    r'\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]'

  
    comeco_pattern = r'^\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:ɾ|ɹ|h|∅r)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]$'


    row['copy_palavra'] = row['palavra']

   
    if re.search(comeco_pattern, row['palavra']):
        row['posição_rótico'] = 'comeco'
        row['copy_palavra'] = re.sub(comeco_pattern, '', row['copy_palavra'])

    elif re.search(meio_pattern_mais, row['palavra']):
        row['posição_rótico'] = 'meio'
        row['copy_palavra'] = re.sub(meio_pattern, '', row['copy_palavra'], count=1)
    
    elif re.search(final_pattern, row['palavra']):
        row['posição_rótico'] = 'final'
        row['copy_palavra'] = re.sub(final_pattern, '', row['copy_palavra'])
    
    else:
        row['posição_rótico'] = ''
    return row

In [83]:
def df_add_duplicated_matches(df):

    # Regular expression pattern
    pattern = r'\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]'

    
    # Find matches for each value in the column
    matches = df['palavra'].str.findall(pattern)
    
    # idenfitfy idexes where there is a match
    match_index = df[matches.apply(lambda x: len(x) > 1)].index
    
    #turn index object to list
    match_list = list(match_index)
    
    i = 0
    for element in match_list:

        element_used = element + i
        new_row_data = df.loc[element_used:element_used]
        new_row_data = new_row_data[['palavra','copy_palavra', 'dimensão_palavra', 'verbo_ou_nome', 'sujeito', 'curso', 'sexo', 'ano', 'cidade']]

        df = concat([df.iloc[:element_used + 1], new_row_data, df.iloc[element_used + 1:]]).reset_index(drop=True)
        i = i + 1
    
    df = df.fillna('') 
    return df


In [88]:
def final_create_posicao_rotico(row):
  
    comeco_pattern = r'^\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:ɾ|ɹ|h|∅r)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(ɾ|ɹ|h|∅r)[^\]]*\]$'
    

    if row['posição_rótico'] == '':


        if re.search(comeco_pattern, row['copy_palavra']) or re.search(meio_pattern_mais, row['copy_palavra']):
            row['posição_rótico'] = 'meio'

        elif re.search(final_pattern, row['copy_palavra']):
            row['posição_rótico'] = 'final'
    return row



In [100]:
def apply_cols(directory_path):
    df = df_find_cases(directory_path)
    
    df['rótico'] = ''
    df['rótico'] = df.apply(lambda row: create_rotico(row,0), axis=1)
    df['apagamento_rótico'] = ''
    df['apagamento_rótico'] = df.apply(create_apagamento_rotico, axis=1)
    # df['rótico_cru'] = ''
    # df['rótico_cru'] = df.apply(lambda row: create_rotico_cru(row,0), axis=1)
    df['verbo_ou_nome'] = ''
    df['verbo_ou_nome'] = df.apply(lambda row: create_verbo_ou_nome(row, 0), axis=1)
    df['dimensão_palavra'] = df['palavra'].apply(create_dimensao_palavra)

    df = df.apply(create_posicao_rotico, axis=1)
    df = df_add_duplicated_matches(df)

    df['rótico'] = df.apply(lambda row: create_rotico(row, 1), axis=1)
    df['apagamento_rótico'] = df.apply(create_apagamento_rotico, axis=1)
    
    df = df.apply(final_create_posicao_rotico, axis=1)
    df = df.drop(columns=['copy_palavra'])

    
    return df


In [101]:
directory_path = "../data/"
df = apply_cols(directory_path)
df

,palavra,sujeito,curso,sexo,ano,cidade,rótico,apagamento_rótico,verbo_ou_nome,dimensão_palavra,posição_rótico
0,pe[ɹ(n)]gunta,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[ɹ],Não,nome,GRANDE,meio
1,cu[ɹ(n)]so,SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[ɹ],Não,nome,MÉDIO,meio
2,convive[∅r(v)],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[∅r],Sim,verbo,GRANDE,final
3,se[ɹ(v)],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[ɹ],Não,verbo,PEQUENO,final
4,melho[∅r(n)],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,[∅r],Sim,nome,MÉDIO,final
...,...,...,...,...,...,...,...,...,...,...,...
1958,es[tʃɪ]ve[ɹ(n)][dʒɪ]s,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[ɹ],Não,nome,GRANDE,meio
1959,acaba[∅r(v)],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[∅r],Sim,verbo,MÉDIO,final
1960,pate[ɹ(n)]no,SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[ɹ],Não,nome,MÉDIO,meio
1961,fo[ɹ(n)][tʃɪ][s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,[ɹ],Não,nome,MÉDIO,meio


In [102]:
# Create a SQLite in-memory database
conn = sqlite3.connect(':memory:')

# Write the DataFrame to the SQLite database
df.to_sql('my_table', conn, index=False, if_exists='replace') 

query = "SELECT * FROM my_table where apagamento_rótico = ''"

result = pd.read_sql_query(query, conn)
result

,palavra,sujeito,curso,sexo,ano,cidade,rótico,apagamento_rótico,verbo_ou_nome,dimensão_palavra,posição_rótico


In [103]:
frequency_table = df['rótico'].value_counts()
percentage_table = df['rótico'].value_counts(normalize=True) * 100

# Combine counts and percentages into a single DataFrame
result_df = pd.DataFrame({'Contagem': frequency_table, 'Porcentagem': percentage_table})
result_df

,Contagem,Porcentagem
rótico,,
[ɹ],1034,52.674478
[∅r],427,21.752420
[ɾ],308,15.690270
[h],194,9.882832


In [108]:
cross_table = pd.crosstab(df['verbo_ou_nome'], df['apagamento_rótico'])
cross_table

apagamento_rótico,Não,Sim
verbo_ou_nome,,
nome,885,56
verbo,651,371


In [109]:
cross_table = pd.crosstab(df['verbo_ou_nome'], df['apagamento_rótico'], normalize='index') * 100
cross_table

apagamento_rótico,Não,Sim
verbo_ou_nome,,
nome,94.048884,5.951116
verbo,63.698630,36.301370


In [111]:
cross_table = pd.crosstab(df['dimensão_palavra'], df['apagamento_rótico'])
cross_table

apagamento_rótico,Não,Sim
dimensão_palavra,,
GRANDE,388,151
MÉDIO,825,260
PEQUENO,323,16


In [112]:
cross_table = pd.crosstab(df['dimensão_palavra'], df['apagamento_rótico'], normalize='index') * 100
cross_table

apagamento_rótico,Não,Sim
dimensão_palavra,,
GRANDE,71.985158,28.014842
MÉDIO,76.036866,23.963134
PEQUENO,95.280236,4.719764


In [113]:
cross_table = pd.crosstab(df['verbo_ou_nome'], df['apagamento_rótico'], margins=True, margins_name='Total', normalize='index') * 100
cross_table

apagamento_rótico,Não,Sim
verbo_ou_nome,,
nome,94.048884,5.951116
verbo,63.698630,36.301370
Total,78.247580,21.752420


In [114]:
cross_table = pd.crosstab(df['curso'], df['apagamento_rótico'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_rótico,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
curso,,,,
DIREITO,477,180,72.602740,27.397260
LETRAS,534,20,96.389892,3.610108
PEDAGOGIA,525,227,69.813830,30.186170


In [115]:
cross_table = pd.crosstab(df['sexo'], df['apagamento_rótico'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_rótico,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
sexo,,,,
HOMEM,727,220,76.768743,23.231257
MULHER,809,207,79.625984,20.374016


In [116]:
cross_table = pd.crosstab(df['ano'], df['apagamento_rótico'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_rótico,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
ano,,,,
PRIMEIRO,805,233,77.552987,22.447013
ÚLTIMO,731,194,79.027027,20.972973


In [117]:
cross_table = pd.crosstab(df['posição_rótico'], df['apagamento_rótico'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_rótico,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
posição_rótico,,,,
final,646,426,60.261194,39.738806
meio,890,1,99.887767,0.112233
